# **라이브러리**

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\재무변수_부도사유_Final.csv', index_col=0)

In [25]:
dataset.head(2)

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,년,key,공시제목,부도
0,(주)CMG제약,58820.0,2000/12,5.0,NaN,UQ,28990.0,21169.0,10541.0,4330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2001.0,588202001,NaN,1.0
1,(주)CMG제약,58820.0,2001/12,5.0,NaN,UQ,20908.0,16359.0,2307.0,1829.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2002.0,588202002,NaN,1.0


# **결측치처리**

In [26]:
drop_feature = ['소속코드','상장폐지일','감사의견코드','년','key','공시제목']

In [27]:
dataset = dataset.drop(drop_feature, axis=1)
dataset

,회사명,거래소코드,회계년도,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),단기투자자산(요약),단기예금(요약),단기매매증권(요약),...,PCR(Price cash-flow ratio)(최저),PSR(Price sales ratio)(최고),PSR(Price sales ratio)(최저),기업가치(EV),EBITDA,EBITDA/매출액,EBITDA/금융비용,EBITDA/평균발행주식수,EV/EBITDA,부도
0,(주)CMG제약,58820.0,2000/12,28990.0,21169.0,10541.0,4330.0,4552.0,4539.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,(주)CMG제약,58820.0,2001/12,20908.0,16359.0,2307.0,1829.0,18.0,0.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,(주)CMG제약,58820.0,2002/12,15401.0,9880.0,1436.0,355.0,246.0,240.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,(주)CMG제약,58820.0,2003/12,11063.0,8534.0,1083.0,2.0,381.0,380.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,(주)CMG제약,58820.0,2004/12,5179.0,3893.0,445.0,11.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27352,흥구석유(주),24060.0,2016/12,79137.0,14794.0,12274.0,1440.0,5065.0,0.0,0.0,...,35.62,0.43,0.28,55934.79,1348.52,0.94,10.71,89.90,41.48,1.0
27353,흥구석유(주),24060.0,2017/12,79912.0,15880.0,13124.0,1588.0,5058.0,0.0,0.0,...,24.31,0.34,0.24,46036.63,1147.56,0.74,9.02,76.50,40.12,1.0
27354,흥구석유(주),24060.0,2018/12,75206.0,12515.0,11029.0,4017.0,5.0,0.0,0.0,...,7.82,0.40,0.24,36557.57,1451.79,0.87,16.83,96.79,25.18,1.0
27355,흥구석유(주),24060.0,2019/12,83661.0,22620.0,19701.0,2318.0,11472.0,0.0,0.0,...,51.31,0.86,0.25,78081.78,1009.25,0.67,507.93,67.28,77.37,1.0


In [28]:
dataset.isnull().sum()

회사명                  0
거래소코드                0
회계년도                 0
자산총계(요약)           333
유동자산(요약)           333
                  ... 
EBITDA/매출액        8885
EBITDA/금융비용       8885
EBITDA/평균발행주식수    8885
EV/EBITDA         9602
부도                   0
Length: 265, dtype: int64

## **결측치처리 알고리즘**

In [36]:
cpylist = dataset['회사명'].unique()

for i in tqdm(range(len(cpylist))):
    dataset.loc[dataset['회사명'] == cpylist[i]] = dataset.loc[dataset['회사명'] == cpylist[i]].fillna(method='pad') # 전년 데이터로 대체
    dataset.loc[dataset['회사명'] == cpylist[i]] = dataset.loc[dataset['회사명'] == cpylist[i]].fillna(method='bfill') # 후년 데이터로 대체
    dataset.loc[dataset['회사명'] == cpylist[i]] = dataset.loc[dataset['회사명'] == cpylist[i]].fillna(0) # 그래도 처리못하는 데이터는 없으면 0으로 대체
    

100%|██████████| 2020/2020 [12:03<00:00,  2.79it/s]


In [38]:
dataset.isnull().sum()

회사명               0
거래소코드             0
회계년도              0
자산총계(요약)          0
유동자산(요약)          0
                 ..
EBITDA/매출액        0
EBITDA/금융비용       0
EBITDA/평균발행주식수    0
EV/EBITDA         0
부도                0
Length: 265, dtype: int64

## **데이터 저장**

In [41]:
#dataset.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\데이터정리중\결측지마무리.csv')